# Introduction

Interactive maps are an essential part of Exploratory Data Analysis (EDA). 
In this talk, I will introduce to you [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) Python library, which enables interactive geospatial data visualization in Jupyter Notebook.

`ipyleaflet` is a bridge between the worlds of Jupyter Notebooks and the open-source JavaScript library for interctive maps [leaflet.js](https://leafletjs.com/).

# Jupyter Widgets

Jupyter Interactive Widgets are “special objects” that can be instantiated by the user in their code and results in a counterpart component being created in the front-end.
The core [ipywidgets](https://pypi.org/project/ipywidgets/) package provides a collection of controls that Jupyter users can use to build simple UIs as part of their notebooks (sliders, buttons, dropdowns, layout components).

Apart from collection of controls, it provides a framework upon which a large ecosystem of components has been built, allowing Notebook authors to capture user inputs in very diverse ways.

### Simple Slider using ipywidgets

In [ ]:
from ipywidgets import IntSlider

slider = IntSlider()

slider

In [ ]:
slider.value

In [ ]:
slider.value = 3

In [ ]:
slider

# Ipyleaflet

ipyleaflet is a Jupyter - [leaflet.js](https://leafletjs.com/) bridge, bringing mapping and visualisation capabilities to the notebook and JupyterLab.
Built as a bridge between the `leaflet.js` package and Jupyter, the ipyleaflet API maps to that of `leaflet.js`, bringing most of the core features of the package to Jupyter, and enabling a few popular `leaflet.js` extensions.

In [ ]:
from ipyleaflet import Map

In [ ]:
m = Map(center=[19.1334, 72.9133], zoom=6)
m

In [ ]:
m.zoom

In [ ]:
m.zoom=10

## Basemaps

In [ ]:
from ipyleaflet import Map, basemaps

center = [19.1334, 72.9133]
zoom = 5

m = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=center, zoom=zoom)
m

# Layers
Add data as layer on top of basemap

### Marker
You usually want to have your data on top of base map. 
Lets see simplest example. 

Let us say we want to add a point marker at the centre of our map. 
We can use object of the `Marker` class of ipyleaflet.


In [ ]:
from ipyleaflet import Map, Marker

center = [19.1334, 72.9133]
zoom = 15

m = Map(center=center, zoom=zoom)

marker = Marker(location=center, draggable=False)
m.add_layer(marker);
m

### GeoJSON Layer
GeoJSON standard - https://geojson.org/

In [ ]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json

# Map and label widgets

m = ipyl.Map(center=[53.90, 27.42], zoom=3)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


# Create GeoJSON layer

with open('./countries.geo.json') as f:
    data = json.load(f)
    
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'green',
        'weight': 1,
        'fillColor': 'green',
        'fillOpacity': 0.5
    }
    
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})
m.add_layer(layer)


# Add a callback for GeoJSON layer

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = properties['name']

    
layer.on_hover(hover_handler)


# Display

ipyw.VBox([m, label])

### Choropleth Layer
A choropleth map is a type of thematic map in which a set of pre-defined areas is colored or patterned in proportion to a statistical variable that 
represents an aggregate summary of a geographic characteristic within each area, such as population density or per-capita income.

In [ ]:
import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear


with open('us-states.json') as fjson:
    geo_json_data = json.load(fjson)

unemployment_df = pd.read_csv('US_Unemployment_Oct2012.csv')

unemployment =  dict(zip(unemployment_df['State'].tolist(), unemployment_df['Unemployment'].tolist()))

In [ ]:
unemployment_df.head()

In [ ]:
linear.YlOrRd_04

In [ ]:
layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=unemployment,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8})

m = ipyleaflet.Map(center = (43,-100), zoom = 4)
m.add_layer(layer)
m

### Marker Cluster

In [ ]:
from ipyleaflet import Map, Marker, MarkerCluster
import geopandas

cities = geopandas.read_file("zip://./geopandas_cities.zip")

In [ ]:
cities.shape

In [ ]:
cities.head()

In [ ]:
m = Map(center=(42.5, -41.6), zoom=2)

m.add_layer(MarkerCluster(
    markers=[Marker(location=geolocation.coords[0][::-1]) for geolocation in cities.geometry])
    )
m

### GeoData Layer
`GeoData` is an ipyleaflet class that allows you to visualize a GeoDataFrame on the Map.

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json

countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
rivers = geopandas.read_file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines.zip")

In [ ]:
countries.head()

In [ ]:
rivers.head(2)

In [ ]:
m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = countries,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

rivers_data = GeoData(geo_dataframe = rivers,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Rivers')

m.add_layer(rivers_data)
m.add_layer(geo_data)
m.add_control(LayersControl(position='topright'))

m

# Controls

### Widget Control

In [ ]:
from ipyleaflet import Map, basemaps, WidgetControl
from ipywidgets import IntSlider, jslink

m = Map(center=(19.1334, 72.9133), zoom=12, basemap=basemaps.Stamen.Terrain)

zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=7)

jslink((zoom_slider, 'value'), (m, 'zoom'))

widget_control = WidgetControl(widget=zoom_slider, position='topright')

m.add_control(widget_control)

m

In [ ]:
m.zoom

### Search Control

In [ ]:
from ipyleaflet import Map, SearchControl, Marker, AwesomeIcon

m = Map(zoom=3, center=[19.1646, 72.8493])

marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

search = SearchControl(position="topleft", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=14,
                       property_name='display_name',
                       marker=marker
                      )
m.add_control(search)

m

### Split Map Control

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl

m = Map(center=(42.6824, 365.581), zoom=5)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11")
left_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m